In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

In [2]:
def get_preprocessed_df(df: pd.DataFrame, base_imbalance_orderbook_level: int) -> pd.DataFrame:
    df['MidPrice'] = (df["AskPrice1"]+df["BidPrice1"])/2
    df['Return'] = (-df["MidPrice"]+df["MidPrice"].shift(-1)) / df["MidPrice"]

    pbid = df["BidPrice1"] - df[f"BidPrice{base_imbalance_orderbook_level}"]
    pask = df[f"AskPrice{base_imbalance_orderbook_level}"] - df["AskPrice1"]
    df["BaseImbalance"] = (pbid-pask)/(pbid+pask)

    df=df.dropna(subset=['Return', 'BaseImbalance'])
    df = df[df['Return'] != 0]
    df = df[['Timestamp', 'BaseImbalance', 'Return']]
    df['TimestampDiff'] = -df['Timestamp'].diff(-1)

    return df


In [3]:
files = []
for filename in os.listdir('/home/davide/Desktop/phd/bitfinex-api-py/data/orderbook_changes'):
    if filename.endswith('.tsv'):
        files.append('/home/davide/Desktop/phd/bitfinex-api-py/data/orderbook_changes' + filename)
dfs = []
for file in files:
    df = pd.read_csv('/home/davide/Desktop/phd/bitfinex-api-py/data/orderbook_changes/orderbook_changes_1705074061033.tsv', sep='\t')
    df = get_preprocessed_df(df, 8)
    dfs.append(df)
df = pd.concat(dfs)
df.head()

,Timestamp,BaseImbalance,Return,TimestampDiff
13,1705070763156,0.000000,0.000088,242.0
14,1705070763398,-0.435897,0.000155,231.0
15,1705070763629,-0.480000,-0.000088,1.0
16,1705070763630,-0.277778,0.000011,237.0
17,1705070763867,-0.500000,-0.000066,251.0


In [4]:
df[['Return', 'TimestampDiff']].corr('spearman')

,Return,TimestampDiff
Return,1.000000,0.021583
TimestampDiff,0.021583,1.000000


In [5]:
df[['Return', 'TimestampDiff']].corr('pearson')

,Return,TimestampDiff
Return,1.000000,-0.009184
TimestampDiff,-0.009184,1.000000
